In [1]:
import os
import cv2
from tqdm import tqdm

output_path = '/content/drive/My Drive/resized_images'

# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Chỉ định chất lượng nén JPEG (0 đến 100)
jpeg_quality = 90  # Bạn có thể thay đổi giá trị này để điều chỉnh chất lượng ảnh

for filename in tqdm(os.listdir('/content/drive/My Drive/cropped_images')):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join('/content/drive/My Drive/cropped_images', filename)
        img = cv2.imread(image_path)

        if img is not None:
            # Thay đổi kích thước ảnh
            new_width = int(img.shape[1] / 2)
            new_height = int(img.shape[0] / 2)
            resized_img = cv2.resize(img, (new_width, new_height))

            # Lưu ảnh đã thay đổi kích thước dưới dạng JPEG với chất lượng nén
            output_resized_path = os.path.join(output_path, filename.replace('.png', '.jpg').replace('.jpeg', '.jpg'))
            cv2.imwrite(output_resized_path, resized_img, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg_quality])

print("Hoàn tất quá trình resize và chuyển đổi sang JPEG!")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/cropped_images'

In [1]:


import os
import cv2
from tqdm import tqdm
dataset_path = 'secret_test_set'  # Thư mục chứa dataset
output_path = 'secret_test_set_cropped'  # Thư mục trên Google Drive để lưu ảnh đã crop

# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Lặp qua các tệp trong dataset
for filename in tqdm(os.listdir(dataset_path)):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_path, filename)
        img = cv2.imread(image_path)

        # Chuyển đổi sang ảnh xám
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Làm mờ để giảm nhiễu
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Áp dụng ngưỡng nhị phân
        _, binary = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY_INV)

        # Tìm các đường viền
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Khởi tạo các điểm cực trị cho hộp bao
        x_min, y_min = img.shape[1], img.shape[0]
        x_max, y_max = 0, 0

        # Tìm các điểm cực trị qua từng contour
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            x_max = max(x_max, x + w)
            y_max = max(y_max, y + h)

        # Cắt ảnh bằng hộp bao
        if x_min < x_max and y_min < y_max:
            cropped_img = img[y_min:y_max, x_min:x_max]

            # Lưu ảnh đã cắt vào Google Drive
            output_cropped_path = os.path.join(output_path, filename)
            cv2.imwrite(output_cropped_path, cropped_img)

print("Quá trình xử lý và lưu ảnh hoàn tất!")

  9%|▊         | 658/7720 [00:20<03:39, 32.24it/s]


KeyboardInterrupt: 

In [7]:
import os
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm  # Import tqdm

def resize_image(input_path, output_path, target_width, target_height):
    img = Image.open(input_path)

    # Tính toán tỷ lệ
    aspect_ratio = img.width / img.height

    # Tính toán kích thước mới
    if target_width / target_height > aspect_ratio:
        new_width = int(target_height * aspect_ratio)
        new_height = target_height
    else:
        new_width = target_width
        new_height = int(target_width / aspect_ratio)

    # Resize ảnh
    img_resized = img.resize((new_width, new_height), Image.LANCZOS)

    # Tạo một bức ảnh mới với màu trắng
    img_final = Image.new("RGB", (target_width, target_height), (255, 255, 255))  # Màu trắng

    # Tính toán vị trí để chèn ảnh đã resize
    left = (target_width - new_width) // 2
    top = (target_height - new_height) // 2

    # Chèn ảnh đã resize vào bức ảnh mới
    img_final.paste(img_resized, (left, top))

    # Lưu ảnh
    img_final.save(output_path)
# Đường dẫn tới folder chứa ảnh và folder output
input_folder = 'cropped_images'  # Thay đổi theo thư mục của bạn
output_folder = 'fixed_im'  # Thay đổi theo thư mục của bạn

# Tạo folder output nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Lấy danh sách tất cả file ảnh trong folder input
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Resize tất cả ảnh trong folder
for filename in tqdm(image_files, desc="Resizing Images", unit="image"):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    resize_image(input_path, output_path, 512, 64)


Resizing Images: 100%|██████████| 8115/8115 [05:15<00:00, 25.69image/s]
